# 0.0 - Imports

In [23]:
import pandas  as pd
import numpy   as np
import seaborn as sns 
import math

from scipy.stats       import shapiro, bartlett 
from statsmodels.stats import power, proportion, gof

## 0.1 - Helper Functions

##  0.2 - Carregamento dos Dados

In [5]:
df = pd.read_csv('../data/raw/ab_data.csv').copy()
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# 1.0 - Escolha do Método

In [6]:
df1 = df.copy()

- Para solucionar esse problema, vamos utilizar o teste A/B + o Teste de Hipótese

# 2.0 - Design do Experimento

In [9]:
df2 = df1.copy()

In [10]:
df2.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


## 2.1 - Formulação das Hipóteses

**Hipótese Nula:** A página antiga e a página nova têm a mesma taxa de conversão.

**Hipótese Alternativa:** A página com o design novo tem uma taxa de conversão diferente da página atual.(Teste Bicaudal)

**Métrica:** A proporção da quantidade de conversões

**Qual o tipo dos dados?** numéricos.

**Qual o objetivo do teste?** Verificar diferença.

**Que tipo de diferença?** Diferença de proporção.

**Quantos grupos?** São 2 grupos.

Utilizar o Two Proportion Z-Test.
- Verificar as premissas básicas para aplicar o teste (Shapiro, Bartlett's e Independência)

## 2.2 - Parâmetros da Amostra

In [13]:
# ---------------------- Determinação dos Parâmetros Para Dimensionar o Tamanho das Amostras -----------------------

# Nível de Confiança
nivel_confianca = 0.95

# Nivel de Significância
nivel_significancia = 1 - nivel_confianca

# Effect Size
# Conversões das páginas atual e nova.
conversao_pagina_atual = 0.13
conversao_pagina_nova  = 0.15

# Cálculo do effect_size
effect_size = proportion.proportion_effectsize(conversao_pagina_atual, conversao_pagina_nova)

# Poder Estatístico
power_stats = 0.8

## 2.3 - Tamanho da Amostra

In [14]:
# ------------------------------- Cálculo do Tamanho da Amostra -----------------------------------------------

# Tamanho da Amostra
tamanho_amostra = power.zt_ind_solve_power(effect_size=effect_size , 
                                           power=power_stats, 
                                           alpha=nivel_significancia,
                                           alternative='two-sided')

# Reajustes
tamanho_amostra = math.ceil(tamanho_amostra)

print('O tamanho da amostra para o grupo de controle é {}.'.format(tamanho_amostra))
print('O tamanho da amostra para o grupo de tratamento é {}.'.format(tamanho_amostra))
print('O tamanho total da amostra é {}.'.format(tamanho_amostra*2))

O tamanho da amostra para o grupo de controle é 4720.
O tamanho da amostra para o grupo de tratamento é 4720.
O tamanho total da amostra é 9440.


# 3.0 - Coleta e Prepação dos Dados da Amostra

In [15]:
df3 = df2.copy()

## 3.1 - Preparação dos Dados

### 3.1.1 - Dimensão dos Dados

In [16]:
# ------ Verificação da Dimensão dos Dados --------------

print('Número de Linhas: {}.'.format(df3.shape[0]))
print('Número de Colunas: {}.'.format(df3.shape[1]))

Número de Linhas: 294478.
Número de Colunas: 5.


### 3.1.2 - Check NA

In [17]:
# -------------- Verificação de Dados Nulos -------------------
df3.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

### 3.1.3 - Tipagem dos Dados 

In [18]:
# ---------------- Verificação da Natureza dos Dados -------------------
df3.dtypes

user_id          int64
timestamp       object
group           object
landing_page    object
converted        int64
dtype: object

## 3.2 - Coleta dos Dados de Amostra

### 3.2.1 - Duplicidade das Flags

In [19]:
# Verificação para saber se existe usuários com duas flags diferentes.
df_flags = df3[['user_id', 'group', 'landing_page']].groupby(['group', 'landing_page']).count().reset_index()

# renomeando a coluna user_id
df_flags = df_flags.rename(columns={'user_id':'number_users'})

# exibindo o dataframe
df_flags

,group,landing_page,number_users
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


In [20]:
# ----------- Remoção das Duplicidades ------------------------

# dataframe com os usuários duplicados
df_users_duplicates = df3[['user_id', 'group']].groupby('user_id').count().reset_index().query('group > 1')

# colocar os user_id em uma lista.
list_users_duplicates = df_users_duplicates['user_id'].values.tolist()

# Removendo as duplicidades do dataframe principal
df3 = df3[~df3['user_id'].isin(list_users_duplicates)].copy()

# Porcentagem removida
percent = round(len(list_users_duplicates)*2/df2.shape[0]*100, 2)
print('Porcentagem removida: {}%.'.format(percent))

# -------------------- Verificação da remoção das duplicidades --------------------

# Verificação para saber se existe usuários com duas flags diferentes.
df_flags = df3[['user_id', 'group', 'landing_page']].groupby(['group', 'landing_page']).count().reset_index()

# renomeando a coluna user_id
df_flags = df_flags.rename(columns={'user_id':'number_users'})

# exibindo o dataframe
df_flags

Porcentagem removida: 2.64%.


,group,landing_page,number_users
0,control,old_page,143293
1,treatment,new_page,143397


### 3.2.2 - Amostragem Aleatória dos Grupos de Controle e Tratamento

In [21]:
# Amostragem do Grupo de Controle
df_control_sample = df3[df3['group'] == 'control'].sample(n=tamanho_amostra, random_state=42)

# Amostragem do Grupo de Tratamento
df_treatment_sample = df3[df3['group'] == 'treatment'].sample(n=tamanho_amostra, random_state=42)

# Exibição
print('Tamando da amostra do grupo de controle: {}.'.format(df_control_sample.shape[0]))
print('Tamando da amostra do grupo de tratamento: {}.'.format(df_treatment_sample.shape[0]))

Tamando da amostra do grupo de controle: 4720.
Tamando da amostra do grupo de tratamento: 4720.


## 3.3 -  Cálculo da Conversão dos Grupos(Taxa de Conversão)

In [36]:
# ------------------------ Métrica Grupo de Controle --------------------

# Quantidade de Conversões
sales_control = df_control_sample[df_control_sample['converted'] == 1]['converted'].sum()

# Quantidade de Visitas
visit_control = len(df_control_sample)

# Taxa de Conversão do Grupo de Controle
taxa_conversao_controle = round(sales_control/visit_control, 4)

# ------------------------ Métrica Grupo de Tratamento --------------------

# Quantidade de Conversões
sales_treatment = df_treatment_sample[df_treatment_sample['converted'] == 1]['converted'].sum()

# Quantidade de Visitas
visit_treatment = len(df_treatment_sample)

# Taxa de Conversão do Grupo de Controle
taxa_conversao_tratamento = round(sales_treatment/visit_treatment, 4)

# ----------------- Organização dos Dados -------------------------------

# Armazenando os dados na tabela
df_sample = pd.DataFrame({'vendas' : [sales_control, sales_treatment], 
                          'não-vendas': [visit_control - sales_control, visit_treatment-sales_treatment], 
                          'visitas' : [visit_control, visit_treatment]})
# Reajustando os nomes
df_sample.index = ['Grupo Controle', 'Grupo Tratamento']


# ------------ Exibição das Métricas ----------------------

print('Taxa de conversão no grupo de controle: {}%.'.format(taxa_conversao_controle*100))
print('Taxa de conversão no grupo de tratamento: {}%.'.format(taxa_conversao_tratamento*100))

df_sample

Taxa de conversão no grupo de controle: 11.55%.
Taxa de conversão no grupo de tratamento: 12.9%.


,vendas,não-vendas,visitas
Grupo Controle,545,4175,4720
Grupo Tratamento,609,4111,4720


# 4.0 - Teste de Hipóteses

## 4.1 - Determinação do Teste

**1.** Qual o tipo dos dados?
    - Numéricos. Estamos interassados na proporção de conversão.
    
**2.** Qual o tamanho da amostra?
    - 4720
    
**3.** Quantos grupos?
    - 2 Grupos
    
**Teste Escolhido:** Two Proportion Ztest

## 4.2 - Two Proportion Z-Test

In [43]:
# -------- Ajuste dos dados para o teste -----------------------

# Junção das amostras de controle e tratamento.
sales = np.array(df_sample['vendas'])

visit = np.array(df_sample['visitas'])

# realização do teste
zscore, p_valor = proportion.proportions_ztest(sales, visit, alternative='two-sided')

print(f'P-Valor: {p_valor}.')
if p_valor < (nivel_significancia):
    print('Rejeitamos a hipótese nula. Logo, concluímos que há uma diferença entre as taxas de conversões da página.')
else:
    print('Falhamos em rejeitar a hipótese nula. Portanto, concluímos que não há uma diferença entre as taxas de conversões.')

P-Valor: 0.044335957690484505.
Rejeitamos a hipótese nula. Logo, concluímos que há uma diferença entre as taxas de conversões da página.


Em nosso teste, nós encontramos que existe um diferença entre as conversões das páginas. Primeiramente, vamos verificar se a conversão da página nova é maior que a atual. Aqui reformulamos a nossas hipóteses da seguinte forma:

**Hipótese Nula:** A página antiga e a página nova têm a mesma taxa de conversão.

**Hipótese Alternativa:** A página com o design novo tem uma taxa de conversão maior que a página atual.(Teste Unicaudal)


Segue o teste abaixo com as mesmas amostras.

In [44]:
# realização do teste
zscore, p_valor = proportion.proportions_ztest(sales, visit, alternative='smaller')

print(f'P-Valor: {p_valor}.')
if p_valor < (nivel_significancia):
    print('Rejeitamos a hipótese nula. Logo, concluímos que a página nova possui uma taxa de conversão maior que a antiga.')
else:
    print('Falhamos em rejeitar a hipótese nula. Portanto, concluímos que não há uma diferença entre as taxas de conversões.')

P-Valor: 0.022167978845242253.
Rejeitamos a hipótese nula. Logo, concluímos que a página nova possui uma taxa de conversão maior que a antiga.


Sabendo que a página com o design novo possui uma taxa de conversão maior que a página atual, vamos realizar o teste estatistico para determinar o quão melhor é a página com o novo design. Para isso, vamos realizar o One Proportion Z_test sobre os dados da amostra de tratamento. Segue a formulação das hipóteses:

**Hipótese Nula:** A página nova possui uma taxa de conversão de 13.9%.

**Hipótese Alternativa:** A página nova possui uma taxa de conversão menor que 13.9%

In [158]:
zscore, pvalor = proportion.proportions_ztest(sales_treatment, visit_treatment, value=0.139, alternative='smaller')

print('P-valor: {}'.format(pvalor))
if p_valor < 0.05:
    print('Rejeitamos a hipótese nula. Logo, a taxa de conversão é menor que a 13.9%.')
else: 
    print('Falhamos em rejeitar a hipótese nula. Logo, a taxa de conversão é de 13.9%.')

P-valor: 0.020466505038432205
Rejeitamos a hipótese nula. Logo, a taxa de conversão é menor que a 13.9%.


# 5.0 - Respondendo as perguntas do CEO.

**1.** A conversão da nova página é realmente melhor do a conversão da página atual?
    
- Após realizado os testes, concluímos que a página com o novo design é realmente melhor que a página atual. De forma mais detalhada de acordo com os teste, é esperado uma melhora de 0.8% na taxa de conversão da página nova em relação a página antiga, pois a página nova possui uma taxa de conversão maior que atual que é 13% e a taxa de conversão da nova página é menor que 13,9%, segundo os testes estatísticos realizados.

**2.** Qual o potencial de número de vendas que a nova página pode trazer?

- Segundo os dados da empresa, com a página antiga, nós tivemos uma taxa de conversão de 13%, na média, no último ano com a venda de teclado bluetooth, isto é, a cada 100.000 acessos, podemos dizer que foram vendido 13.000 teclados bluetooth, ou seja, temos um faturamento de R$ 58.800.00

 
- Com a nova página atuando, é esperado um lift de 0.8% na taxa de conversão. Logo, para cada 100.000 acessos, podemos dizer que se espera uma venda de 800 teclados bluetooth a mais, ou seja, temos uma estimativa de aumento no faturamento de R$ 3.600.000.


**3.** Qual o faturamento total na venda do teclado bluetooth através da nova página?
    
- Como não temos informações sobre a quantidade média de acessos anualmente, esperasse que a cada 100.000 acessos tenhamos um faturamento total de R$ 62.100.00 com a nova página.
    